## **(Implementation section 3.2.3)This is the script used to create the additional words per song dataset**

Creates a function using the beautful soup API. The function passes the Created URL for the specific songs lyrics. The number of words contained within the lyrics are returned.

In [0]:
import requests
from bs4 import BeautifulSoup
import numpy as np

# Function that takes the custom url and returns the words per song
def function(URL):
	# Requests the webpage using the URL
	r = requests.get(URL, allow_redirects=False)
	#  Extracts the section of html from the webpage
	soup = BeautifulSoup(r.content,'html5lib')

# Initialise an empty array to store the song lyrics 
	lyrics = []
	# Iterates through the html returned from the webpage till the relevant section containing the lyrics is found
	for paragraph in soup.find_all("p",{"id":"songLyricsDiv"}):
		# Appends the lyrics to the empty array
		lyrics.append(paragraph.text)

	# Creates a function to format the lyrics returned. As the lyrics are done line by line in list format,
	# they need to be formatted back into string format.
	def listtostring(s):
		# Intialise an empty string to store the formatted lyrics
		str1 = " "
		return(str1.join(s))
	# Calls the list to string function using the lyrics extracted from the html
	st = listtostring(lyrics) 
	# Splits the individual strings into one large section of text
	st =st.split()

# Returns the fomratted lyrics from the song
	return st

In [0]:
import pandas as pd
# Loads in the million song dataset for the audio and song and artist names
data = pd.read_csv('msd_genre_dataset.csv')

# Extract the song name and artist names for each song
title = data['title']
artist = data['artist_name']

# Initalise an array to store the created URL link
link = []

# Iterate through every song within the dataset and create a link to the website.
# Once the link is created it is stored within the array
for i in range(len(title)):
  # Gets the song title and artist name at the given index. Important to get the name 
  # of the song and artist name in lower case.
	sn = title.loc[i].lower()
#  Formats the spaces with - for the URL
	sn = sn.replace(' ','-')
	an = artist.loc[i].lower()
	an = an.replace(' ','-') 
#  Adds the artist name and song name in the correct format as an extension to the base URL
	temp = an+'/'+sn+'-lyrics/'
	url = "http://www.songlyrics.com/"
  # Create the new URL for that song and artist name 
	temp = url+temp
  # Add the created URL to an empty array
	link.append(temp)

# Creates an empty array to store the index of any songs where the lyrics were incorrect
# or couldnt be returned
incorrect = []
# Creates an empty array to store the number of words for that song
wpm = []
# Iterates through the the number of songs
# *****Demonstration Purposes******
for i in range(10):
# UNCOMMMENT FOR WHEN NOT BEING USED WITHIN THE DEMONSTRATION
# for i in range(len(data['title'])):

  # Calls the function to get the song lyrics using the URL
  t = function(link[i])
  # Checks the length of the lyrics returned is greater than 1
  if len(t) == 0:
    incorrect.append(i)
  # Checks that the lyrics returned do not start with sorry as this demonstrated
  # the lyrics could not be returned
  elif t[0] == 'Sorry,':
    incorrect.append(i)
  # checks that the length of the lyrics were greater than 40 as this indicated
  # short hand was used
  elif len(t) < 40:
    incorrect.append(i)
  # if all the conditions are passed then the number of words along with the song
  # index is added to the array
  else:
    wpm.append([i,len(t)])

***ADDED FOR DEMONSTRATION **** 
Shows the output of the number of words returned for the lyrics that could be located

In [0]:
wpm

[[1, 130], [3, 227], [5, 128], [6, 216], [7, 119]]

In [0]:
# Creates an additional empty column within the dataset to store the words per song value
data['wpm'] = 0
# Iterates through the all the songs that have a words per song value and adds them in the correct location
for i in range (len(wpm)):
  d = wpm[i]
  data['wpm'].iloc[d[0]] = d[1]

In [0]:
# Drops all the data values that did not have a words per song value
for i in range(len(data)):
  if data['wpm'].iloc[i] == 0:
    data = data.drop(data.index[i])

In [0]:
# Stores the dataset as a csv file
from numpy import savetxt
savetxt('wpmdata.csv', data, delimiter=',')